In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import openpyxl
from datetime import datetime

# Função para extrair informações do site e salvar na planilha
def extrair_informacoes(link, linha):
    requisicao = requests.get(link)
    site = BeautifulSoup(requisicao.text, "html.parser")
    
    nome_produto = site.find(class_='product-info__title h2')
    
    preco_elemento = site.find('sale-price')
    if preco_elemento:
        price_text = preco_elemento.get_text()
        if 'Sale price' in price_text:
            preco_produto = price_text.split('Sale price')[-1].strip()
        else:
            preco_produto = price_text.strip()
        
        # Remover caracteres indesejados e formatar o preço
        preco_produto = preco_produto.replace('$', '').replace(',', '').replace('.', ',')
        if ',' in preco_produto and '.' in preco_produto:
            preco_produto = preco_produto.replace(',', '.').replace('.', ',', 1)
        
        # Forçar o tipo de dado para número
        preco_produto = float(preco_produto.replace(',', '.'))
    else:
        preco_produto = None
    
    sku_tag = site.find('variant-sku')
    if sku_tag:
        sku_produto = sku_tag.get_text(strip=True).replace('SKU:', '').strip()
    else:
        sku_produto = None

    div_thumbnail = site.find('button', class_='product-gallery__thumbnail')
    if div_thumbnail:
        img_tag = div_thumbnail.find('img')
        if img_tag:
            image_link = img_tag['src']
        else:
            image_link = None
    else:
        image_link = None

    # Encontrando a próxima linha vazia na planilha
    proxima_linha = sheet.max_row + 1
    
    # Salvando as informações na planilha na próxima linha vazia
    sheet[f'A{proxima_linha}'] = nome_produto.text if nome_produto else ''
    sheet[f'B{proxima_linha}'] = preco_produto if preco_produto else ''
    sheet[f'C{proxima_linha}'] = sku_produto if sku_produto else ''
    sheet[f'D{proxima_linha}'] = image_link
    sheet[f'F{proxima_linha}'] = 'Massive Audio USA'
    
    # Adicionando a data na próxima linha vazia
    sheet[f'E{proxima_linha}'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Definir o formato da célula como número
    if preco_produto:
        sheet[f'B{proxima_linha}'].number_format = '#,##0.00'
        
# Lendo os links da planilha Excel
planilha = pd.read_excel('03.Input_links_Massive_audio.xlsx')

# Carregando a planilha de produtos existente
workbook = openpyxl.load_workbook('02.Output_informacoes_produtos_Massive_audio.xlsx')
sheet = workbook.active

# Verificando se as colunas já existem na planilha
if sheet['A1'].value != 'Nome do Produto':
    sheet['A1'] = 'Nome do Produto'
if sheet['B1'].value != 'Preço do Produto':
    sheet['B1'] = 'Preço do Produto'
if sheet['C1'].value != 'SKU do Produto':
    sheet['C1'] = 'SKU do Produto'
if sheet['D1'].value != 'Imagem do Produto':
    sheet['D1'] = 'Imagem do Produto'   
if sheet['E1'].value != 'Data da Extração':
    sheet['E1'] = 'Data da Extração'
if sheet['F1'].value is None or 'Origem' not in sheet['F1'].value:
    sheet['F1'] = 'Origem'
    
# Iterando sobre cada linha da planilha de links
for idx, row in planilha.iterrows():
    link = row['Link']
    extrair_informacoes(link, idx + 2)  # Começa da linha 2
    
# Salvando a planilha
workbook.save('02.Output_informacoes_produtos_Massive_audio.xlsx')
